# Read Inventory

In [ ]:
import json
import sys
from datetime import datetime
import pandas as pd
from tabulate import tabulate

In [ ]:
with open('record.json', 'r') as file:
  r = file.read()

record = json.loads(r)

In [ ]:
record

# Purchase

### Display the products


In [ ]:
def available_products():
  print("------------------------------   WELCOME!   ----------------------------------\n")
  print("Following products are available in our Store\n")
  print("------------------------------------------------------------------------------")
  prod_df = pd.DataFrame(record).transpose()
  prod_headers = ['PRODUCT ID', 'PRODUCT', 'QUANTITY', 'PRICE', 'PKD/MFG', 'EXPIRY/WARRANTY']

  return (tabulate(prod_df, headers=prod_headers, tablefmt='fancy_grid'))
  

### Add to Sales

In [ ]:
invoice_list = []
sales = {}
def update_sales():
  now = datetime.now()
  dt_str = now.strftime("%d/%m/%Y %H:%M:%S")
  date, time = dt_str.split(" ")
  sales[len(sales) + 1] = {'customer_name': cust_name,'product_id' : product, 'product_name': record[product]['name'], 'qty' : quant, 'amount': record[product]['price'] * quant, 'date': date, 'time': time}
  invoice_list.append(list(sales[len(sales)].values()))

  # Update the Sales.txt
  s_js = json.dumps(sales)

  with open('sales.json', 'w') as file:
    file.write(s_js)

  return sales


### Generate Invoice

In [ ]:
def generate_invoice(invoice_list):
  dt = str(datetime.now().year) + "-" + str(datetime.now().month) + "-" + str(
          datetime.now().day) + "-" + str(datetime.now().hour) + "-" + str(
          datetime.now().minute) + "-" + str(datetime.now().second)
  unique_invoice = str(dt)  # unique invoice
  t = str(datetime.now().year) + "-" + str(datetime.now().month) + "-" + str(datetime.now().day)  # date
  d = str(t)  # date
  u = str(datetime.now().hour) + ":" + str(datetime.now().minute) + ":" + str(datetime.now().second)  # time
  e = str(u)  # time

  with open(unique_invoice + " (" + cust_name + ").txt", "w") as file:
    file.write("=============================================================")
    file.write("\nONLINE STORE\t\t\t\t\t\tINVOICE")
    file.write("\n\nInvoice: " + unique_invoice + "\t\tDate: " + d + "\t\tTime: " + e + "")
    file.write("\n\nName of Customer: " + cust_name.upper() + "")
    file.write("\n=============================================================\n")

    header = ['PRODUCT ID', 'PRODUCT', 'QUANTITY', 'UNIT PRICE', 'TOTAL']
    table = []
    total_amount = 0
    for values in invoice_list: 
      total_amount += int(values[4])
      table.append([str(values[1]), str(values[2]), str(values[3]), str(record[values[1]]['price']), str(values[4])])
    file.write(tabulate(table, headers=header))

    file.write("\n-------------------------------------------------------------")
    file.write("\n\t\t\t\t\t\t\t\t Your payable amount is: " + str(total_amount))
    file.write("\n-------------------------------------------------------------")
    file.write("\n\n\t\t\t\t\t\t\tThank you " + cust_name.upper() + " for your shopping.\n\t\t\t\t\t\t\t\t\t\t\t See you again!")
    file.write("\n=============================================================")
  
  return unique_invoice
  

### Buying the product

In [ ]:
print(available_products())

cust_name = input("\nEnter your Name: ")
print(f"\nHello {cust_name}! Please select product as per your choice.\n")

flag = "Y"

while flag.upper() == "Y":

  product = input("Enter the Product ID: ")

  if product not in record:
    print("\nSorry! Product currently not available.")
    flag = input(cust_name + ", do you want to continue shopping?(Y/N)")

  else:
    quant = int(input("Enter the Quantity: "))

    if quant > record[product]['qty']:

      if record[product]['qty'] == 0:
        print(f"Sorry! {record[product]['qty']} stock left.")
      else:
        print(f"Sorry! Only {record[product]['qty']} stocks left.")
      flag = input(cust_name + ", do you want to continue shopping?(Y/N)")


    else:

      print("\n--------------------------------------------")
      print(f"Product: {record[product]['name']}")
      print(f"Price: {record[product]['price']}")
      print(f"Billing Amount: {record[product]['price'] * quant}")
      print("--------------------------------------------\n")

      record[product]['qty'] = record[product]['qty'] - quant
      update_sales()

      flag = input(cust_name + ", do you want to buy more products?(Y/N)")

unique_invoice = generate_invoice(invoice_list)

print("\n--------------------------------------------\n")
print("Thank you for shopping from our store!!\n")
print("Please check your invoice for your shopping details,")
print(f"Which we have created as '{unique_invoice} ({cust_name}).txt' file.")
print("\n--------------------------------------------\n")

invoice_list.clear()


# Update Inventory after purchase

In [ ]:
r_js = json.dumps(record)

with open('record.json', 'w') as file:
  file.write(r_js)

r_js